## Intel Scene Feature Extractor

**Objective:** Get the features of the Intel Scene images using ResNet50.

**Steps:**
 * Load ResNet50 weights
 * For the training images, generate the HDF5 file with features extracted from ResNet50

## Load ResNet50

In [1]:
from keras.applications import ResNet50

Using Theano backend.


In [2]:
model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

## Get Training Files

In [3]:
from my_utils.datasets import DataSetLoader
import pandas as pd

In [4]:
IMAGE_DIR = 'dataset/train-scene classification'
TRAIN_IMAGE_LOC = IMAGE_DIR+'/train'

In [5]:
train_csv_df = pd.read_csv(IMAGE_DIR+'/train.csv')

In [6]:
train_csv_df.shape

(17034, 2)

In [7]:
DataLoader = DataSetLoader()

In [8]:
training_files = DataLoader.get_filename_list(TRAIN_IMAGE_LOC, list(train_csv_df['image_name']))

In [9]:
training_files[0:5]

['dataset/train-scene classification/train/0.jpg',
 'dataset/train-scene classification/train/1.jpg',
 'dataset/train-scene classification/train/2.jpg',
 'dataset/train-scene classification/train/4.jpg',
 'dataset/train-scene classification/train/7.jpg']

## Transform the labels

In [10]:
train_csv_df.head()

,image_name,label
0,0.jpg,0
1,1.jpg,4
2,2.jpg,5
3,4.jpg,0
4,7.jpg,4


In [11]:
train_labels = train_csv_df['label']

In [12]:
train_labels[0:4]

0    0
1    4
2    5
3    0
Name: label, dtype: int64

In [13]:
from sklearn.preprocessing import LabelEncoder

In [14]:
le = LabelEncoder()

In [15]:
labels = le.fit_transform(train_labels)

In [16]:
labels[0:4]

array([0, 4, 5, 0])

In [17]:
le.classes_

array([0, 1, 2, 3, 4, 5])

### Class labels
'buildings' -> 0,
'forest' -> 1,
'glacier' -> 2,
'mountain' -> 3,
'sea' -> 4,
'street' -> 5

In [18]:
labelNames = ['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']

## Configurations

In [19]:
#NUM_FILES = len(training_files)
NUM_FILES = 68
BATCH_SIZE = 34

In [20]:
NUM_FILES

68

In [21]:
BATCH_SIZE

34

## Create HDF5 Database

In [22]:
from my_utils.io import HDF5DataSetWriter

In [23]:
#resnet50 -> predict returns vector of size 2048
db = HDF5DataSetWriter( (NUM_FILES, 2048), "./output/IntelSceneExtractedFeatures.hdf5", dataKey="features")

## Extract Features

In [24]:
import cv2
import numpy as np
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import img_to_array, load_img

In [25]:
# go over all images
for i in np.arange(0, NUM_FILES, BATCH_SIZE):
    
    # process them in terms of batches
    batchImages = []
    
    batchFileList = training_files[i: i+BATCH_SIZE]
    batchLabelList = train_labels[i: i+BATCH_SIZE]
    print('outer for loop : {} '.format(i))
    
    for fileName in batchFileList:
        # load the file
        img = cv2.imread(fileName)
        img = cv2.resize(img, (224,224), interpolation=cv2.INTER_AREA)
        
        # convert to array
        img = img_to_array(img)
        
        #before preprocess, expand the dim.
        img = np.expand_dims(img, axis=0)
        img = preprocess_input(img)  # for resnet50
        
        batchImages.append(img)
    
    #process the batch images
    batchImages = np.vstack(batchImages)
    
    # extract the features for the batch
    extracted_features = model.predict(batchImages, batch_size=BATCH_SIZE)
    
    ## separate out the features for each image.
    ## resnet50 spits out vector of 2048 for each image
    extracted_features = extracted_features.reshape((extracted_features.shape[0], 2048) )
    
    # add to db
    db.add(extracted_features, batchLabelList)

db.close()

outer for loop : 0 
outer for loop : 34 
self.idx: 0
i : 68
